# Dekoratory

In [27]:
import functools

def shouter(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('-- Before call: ', func.__name__)
        result = func(*args, **kwargs)
        print('-- After call: ', func.__name__)
        return result
    
    return wrapper

In [28]:
@shouter
def foo(name: str):
    """foo function"""
    print(f"foo-{name}")

In [29]:
foo("Hello")

-- Before call:  foo
foo-Hello
-- After call:  foo


In [30]:
foo.__doc__

'foo function'

In [31]:
foo.__name__

'foo'

In [32]:
foo = shouter(foo)

In [33]:
foo("foo with wrapper")

-- Before call:  foo
-- Before call:  foo
foo-foo with wrapper
-- After call:  foo
-- After call:  foo


In [39]:
from typing import Any, Callable

def disable(func: Callable[..., Any]) -> Callable[..., Any | None]:
    @functools.wraps(func)
    def _disable(*args, **kwargs):
        pass
    return _disable

In [40]:
@disable
def bar(n):
    print(f"bar({n})")

In [41]:
bar(42)

## Dekoratory z parametrem

In [49]:
def tag(tagname: str): 
    print(f"creation of tag: {tagname}")
    def tag_decorator(func):
        print(f"magic happens for {func.__name__}")
        @functools.wraps(func)
        def _tag_wrapper(*args, **kwargs):
            tag_before = f"<{tagname}>"
            tag_after = f"</{tagname}>"
            return tag_before + func(*args, **kwargs) + tag_after
        return _tag_wrapper
    return tag_decorator

In [56]:
@tag("h1")
@shouter
def get_html(text):
    return text

creation of tag: h1
magic happens for get_html


In [57]:
get_html("Hello")

-- Before call:  get_html
-- After call:  get_html


'<h1>Hello</h1>'

In [58]:
get_html.__name__

'get_html'

In [53]:
get_html = tag("div")(get_html)

creation of tag: div
magic happens for get_html


In [48]:
get_html("Bye")

'<div><h1>Bye</h1></div>'

## Implementacja dekoratorów za pomocą klasy

In [91]:
class Debug:
    def __init__(self, function: Callable) -> None:
        print(f"Create instance of Debug {function}:{type(function)}")
        self.function = function
        functools.update_wrapper(self, function) # the same as wraps for classic decorator

    def __call__(self, *args, **kwargs):
        print(f"self.function{self.function}:{type(self.function)}")
        result = self.function(*args, **kwargs)
        name = self.function.__name__

        print(f"{name}({args!r}, {kwargs!r}): {result}")

In [92]:
@Debug
def add(a, b):
    return a + b

Create instance of Debug <function add at 0x000001BE6C0F0F40>:<class 'function'>


In [93]:
add(1, 2)

self.function<function add at 0x000001BE6C0F0F40>:<class 'function'>
add((1, 2), {}): 3


In [94]:
add = Debug(add)

Create instance of Debug <__main__.Debug object at 0x000001BE6C00D310>:<class '__main__.Debug'>


In [95]:
add(6, 7)

self.function<function add at 0x000001BE6C0F0F40>:<class 'function'>
add((6, 7), {}): 13


In [96]:
type(add)

__main__.Debug

In [97]:
add

## Rejestracja przy pomocy dekoratorów

In [60]:
import collections


class EventRegistry:
    def __init__(self) -> None:
        self.registry = collections.defaultdict(list)

    def on(self, *events): 
        """decorator for registrations"""
        def _on(function: Callable) -> Callable:
            for event in events:
                self.registry[event].append(function)
            return function
        return _on
    
    def fire(self, event: str, *args, **kwargs):
        for function in self.registry[event]:
            function(*args, **kwargs)

In [65]:
events = EventRegistry()

@events.on('start', 'success')
def start(service_name: str):
    print(f'Starting service {service_name}')

@events.on('error')
def teardown():
    print('Shutting down system')

events.fire('start', "Start me UP")
events.fire('error')

Starting service Start me UP
Shutting down system


## Properties

In [104]:
class IntProperty:
    def __init__(self, fget=None, fset=None, fdel=None) -> None:
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
    
    def __get__(self, obj, owner_cls):
        if obj is None:
            return self
        elif self.fget:
            return self.fget(obj)

    def __set__(self, obj, value):
        if not isinstance(value, int):
            raise ValueError("Value must be int")
        self.fset(obj, value)

    def __delete__(self, obj):
        self.fdel(obj)

    def getter(self, fget):
        return IntProperty(fget, self.fset, self.fdel)

    def setter(self, fset):
        return IntProperty(self.fget, fset, self.fdel)
    
    def deleter(self, fdel):
        return IntProperty(self.fget, self.fset, fdel)

In [105]:
class Person:

    def __init__(self, age) -> None:
        self.__age = age

    @IntProperty
    def age(self):
        return self.__age
    
    # how it works
    # age = IntProperty(age) # now age is descriptor with only __get__
    
    @age.setter 
    def _age(self, new_age):
        self.__age = new_age

    # age = age.setter(fset=_age) # # now age is descriptor with __get__ & __set__


In [106]:
p = Person(20)

In [107]:
p.age = "text"

ValueError: Value must be int

## Dekoracja klas

In [117]:
def sort_by_attribute(attr):
    def _sort_by_attribute(cls):
        def __lt__(self, other):
            return getattr(self, attr) < getattr(other, attr)
        
        def __eq__(self, other):
            return getattr(self, attr) == getattr(other, attr)
        
        # injecting special function to the class
        cls.__lt__ = __lt__
        cls.__eq__ = __eq__

        return cls
    
    return _sort_by_attribute

In [121]:
@sort_by_attribute('name')
class Gadget:
    def __init__(self, id, name):
        self.id = id
        self.name = name
    
    def __repr__(self) -> str:
        return f"Gadget({self.name}, {self.id})"
    
# Gadget = sort_by_attribute('name')(Gadget)

In [122]:
gadgets = [Gadget(1, "ipad"), Gadget(5, "smartwatch"), Gadget(3, "a gadget"), Gadget(665, "laptop")]

In [123]:
sorted(gadgets)

[Gadget(a gadget, 3),
 Gadget(ipad, 1),
 Gadget(laptop, 665),
 Gadget(smartwatch, 5)]

In [124]:
def add_id(decorated_class):
    setattr(decorated_class, '__id_gen', 0)
    print(f'Adding __id_gen to class {decorated_class.__name__}')
    original_init = decorated_class.__init__

    def __init__(self, *args, **kwargs):
        decorated_class.__id_gen += 1
        self.id = decorated_class.__id_gen
        print(f'Adding id={self.id} to object {self}')
        original_init(self, *args, **kwargs)

    decorated_class.__init__ = __init__
    return decorated_class

In [125]:
@add_id
class Person:
    
    def __init__(self, name):
        self.name = name

Adding __id_gen to class Person


In [126]:
p1 = Person("John")

Adding id=1 to object <__main__.Person object at 0x000001BE6C0EA6D0>


In [127]:
p1.id

1

In [128]:
p2 = Person("Eve")

Adding id=2 to object <__main__.Person object at 0x000001BE6C115B50>


In [129]:
p2.id

2

In [130]:
Person.__id_gen

2